# Calibration Tutorial - Crane, OR - Irrigated Flux Plot

## Step 3: Calibrate

Now we see if we are able to improve the model's performance through calibration.

The main calibration tool used in SWIM (PEST++) has been developed over many years by many clever and diligent developers. They've done us a huge favor by writing great documentation that covers the highly varied functionality of PEST++. It's worth it to check out the materials, which will serve those looking for a cursory look, all the way to those that want a deep dive. Several stand-out resources to refer to are the following:

1. The PEST Manual 4th. Ed., Doherty, J., 2002.: https://www.epa.gov/sites/default/files/documents/PESTMAN.PDF. This treats the use of PESTS++ predecessor PEST, but does a great job explaining how we might estimate parameters given observations and a model.
2. The GMDSI tutorial notebooks. These are applications of PEST++ using the groundwater modeling software MODFLOW and the modern Python inteface to PEST++, pyemu: https://github.com/gmdsi/GMDSI_notebooks
3. The PEST++ User's Manual (https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md).
4. Calibration and Uncertainty Analysis for Complex Environmental Models. Doherty, J., 2015. See https://pesthomepage.org/pest-book.

Note: We are not using the flux data for calibration, it's only for validation. We'd probably get a pretty good SWIM model using the flux data to tune the model, but we're interested in the broad applicability of the method, and thus only use the widely-available SSEBop and SNODAS data.

### 1. PEST++ Installation

The PEST++ developers do a great job describing the installation process, so we won't cover it here.

Get the latest release of PEST++ for your operating system: https://github.com/usgs/pestpp/releases

Follow the installation instructions: https://github.com/usgs/pestpp/blob/master/documentation/cmake.md

### 2. Setup the calibration files

In order to use PEST++, we need to run through what is, perhaps, a common loop in model calibration:

1. Intialize the model with intial conditions.
2. Run the model, write the results.
3. Compare the results to observations.
4. Propose a new, hopefully better, set of parameters.
5. Run the model with the new parameters, write the results.
6. Repeat 3 - 5.

And so on, until we are satisfied with the performance of the model. 

The purpose of the SWIM calibration approach and this tutorial is to set up a system where the model and the calibration software can operate with minimal interaction. All we need SWIM to do is take the proposed parameters and use them in a model run, and write the results in a convenient format in a convenient place. All we need the calibration software to do is to compare the model results to observations, determine how to tweak the parameters we've told it are 'tunable', and write a new parameter proposal in a convenient format in a convenient place. If we succeed in building such a system, and have maintained independence between the calibration software and the model, we should be able to make changes to one and not need to make changes to the other. In theory, this objective makes development easier.

The `calibration` package in SWIM contains software to build what we need to do this with three modules:

1. `build_pp_files.py` uses several functions to build the files that control PEST++ behavior:
   - The function `build_pest` builds the main `.pst` control file, which defines the eight tunable SWIM model parameters `'aw', 'rew', 'tew', 'ndvi_alpha', 'ndvi_beta', 'mad', 'swe_alpha'`, and `'swe_beta'`. These are three soil water holding capacity parameters (`'aw', 'rew', 'tew'`), the coefficients that control the relationship between remote-sensing-based NDVI and the model transpiration rate parameter `Kcb` (`'ndvi_alpha', 'ndvi_beta'`), the control on when soil water deficit begins to impact transpiration rate (`'mad'`), and the two coefficients that determine the melting rate of snow (`'swe_alpha'`, `'swe_beta'`). The `.pst` file also contains the observation data, which we have derived from SNODAS (SWE) and SSEBop (ETf). Further, the file contains estimates of the noise we believe is in the data. Finally, the `.pst` points to the main Python file that will be used to call the `pestpp-ies` command, the function that runs the PEST++ implementation of Iterative Ensemble Smoother, the algorithm we'll use.
2. `custom_forward_run.py` has a single, simple function (`run`) that uses a system call to execute a SWIM script that runs the model, much like how we've run it ouselves previously. You will need to modify `custom_forward_run.py` to enter your machine's path.
3. `run_pest.py` is the module that we launch, and that starts PEST++ running. This will also need to be modified to use your machine's path.

The actual flow of code execution during calibration is a little confusing, because we use a Python script (`run_pest.py`) to run a command line executable (`'pestpp-ies'`), which itself then executes `custom_forward_run.py` to finally run our Python SWIM code! I know!


In [1]:
import json
import os
import sys
from datetime import datetime

from tqdm import tqdm
import numpy as np
import pandas as pd
import geopandas as gpd

root = os.path.abspath('../../..')
sys.path.append(root)

from prep.prep_plots import preproc
from swim.config import ProjectConfig

from calibrate.build_pp_files import get_pest_builder_args, initial_parameter_dict
from calibrate.build_pp_files import build_pest, build_localizer, write_control_settings
from calibrate.run_pest import run_pst

Let's just build a `dict` with the paths we're going to need:

In [2]:
# set up our work space directories
project_ws = os.path.join(root, 'tutorials', '3_Crane')
data = os.path.join(project_ws, 'data')

# for convenience, we put all the paths we'll need in a dict
PATHS = {'prepped_input': os.path.join(data, 'prepped_input.json'),
         'plot_timeseries': os.path.join(data, 'plot_timeseries'),
         'ini_path': os.path.join(data, 'tutorial_config.toml'),
         '_pst': os.path.join(project_ws, 'pest', '3_Crane.pst'),
         'exe_': 'pestpp-ies',
         'p_dir': os.path.join(project_ws, 'pest'),
         'm_dir': os.path.join(project_ws, 'master'),
         'w_dir': os.path.join(project_ws, 'workers'),
         'obs': os.path.join(project_ws, 'obs'),
         'mult':  os.path.join(project_ws, 'pest', 'mult'),
         'python_script': os.path.join(root, 'calibrate', 'custom_forward_run.py')}

if not os.path.isdir(PATHS['obs']):
    os.makedirs(PATHS['obs'], exist_ok=True)

In [3]:
# write the observed data to files within project workspace (tutorial directory)
shapefile_path = os.path.join(data, 'gis', 'flux_fields.shp')
gdf = gpd.read_file(shapefile_path)
FEATURE_ID = 'field_1'

# use the following for a full station network extract:
# stations = gdf[FEATURE_ID].tolist()

# use this for just Crane
stations = ['S2']

# We need to specify the model period, which is in the config
config = ProjectConfig()
config.read_config(PATHS['ini_path'], project_ws)
start=datetime.strftime(config.start_dt, '%Y-%m-%d')
end=datetime.strftime(config.end_dt, '%Y-%m-%d')
print(start, end)

preproc(stations, PATHS['plot_timeseries'], project_ws, start=start, end=end)



Config: /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/data/tutorial_config.toml
CALIBRATION ON
FORECAST OFF
1996-01-01 2022-12-31

S2
preproc ETf mean: 0.49
preproc ETa mean: 1.28
preproc SWE mean: 0.00

Prepped 1 fields input


Good, now for the `.pst` control file.

The function `build_pest.py` will erase the existing pest directory if there is one! It will also copy everything from `project_ws` into the `pest` directory, which is nice because it will only manipulate copies after that. The function builds the flux.pst file, which is the only argument needed at this time to run PEST++ on the problem. Note that during the processing of the ETf data, we wrote an e.g., `etf_inv_irr_ct.csv` table that simply marked the image capture dates. In build_pest(), the observations are given weight 1.0 on these dates, and weight 0.0 on non-capture dates, sp we don't use interpolated ETf values for calibration. The idea here is to only evaluate the objective function on capture dates to give the model the freedom to behave like a soil water balance model on in-between dates.

Good. Now, let's build the `.pst` control file for our calibration project. 

Build the `dict` that holds the initial values, file location, upper and lower bounds, and table index for each of the parameters we're tuning:

In [4]:
dct_ = get_pest_builder_args(project_ws, PATHS['prepped_input'], PATHS['plot_timeseries'], start=start, end=end)
# update the dict with the location of 'custom_forward_run.py'
dct_.update({'python_script': PATHS['python_script']})
dct_

{'targets': ['S2'],
 'inputs': ['/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/data/input_timeseries/S2_daily.csv'],
 'etf_obs': {'file': ['obs/obs_etf_S2.np'], 'insfile': ['etf_S2.ins']},
 'swe_obs': {'file': ['obs/obs_swe_S2.np'], 'insfile': ['swe_S2.ins']},
 'pars': OrderedDict([('aw_S2',
               {'file': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/params.csv',
                'initial_value': 100.0000014901161,
                'lower_bound': 15.0,
                'upper_bound': 900.0,
                'pargp': 'aw',
                'index_cols': 0,
                'use_cols': 1,
                'use_rows': 0}),
              ('rew_S2',
               {'file': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/params.csv',
                'initial_value': 3.0,
                'lower_bound': 2.0,
                'upper_bound': 6.0,
                'pargp': 'rew',
                'index_cols': 0,
                'use_cols': 1,
                '

In [5]:
# Build the pest control file
# It will copy everything from the project_ws into a new 'pest' directory

# if you re-run this code, you will have to re-run the above cell, as well
build_pest(project_ws, PATHS['p_dir'], **dct_)

2025-01-05 22:38:58.464203 starting: opening PstFrom.log for logging
2025-01-05 22:38:58.464376 starting PstFrom process
2025-01-05 22:38:58.464420 starting: setting up dirs
2025-01-05 22:38:58.464635 starting: removing existing new_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest'
2025-01-05 22:38:58.563176 finished: removing existing new_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest' took: 0:00:00.098541
2025-01-05 22:38:58.563233 starting: copying original_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane' to new_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest'
2025-01-05 22:38:59.095167 finished: copying original_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane' to new_d '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest' took: 0:00:00.531934
2025-01-05 22:38:59.095612 finished: setting up dirs took: 0:00:00.631192
2025-01-05 22:38:59.095702 starting: adding constant type m style parameters f

See that all the data from `project_ws` are now copied to the 'pest' directory at `swim-rs/3_Crane/pest`, including the data folder, the other steps to this tutorial, etc. We also see the new files that were built:

In [6]:
original_files = [f for f in sorted(os.listdir(PATHS['p_dir'])) if os.path.isfile(os.path.join(PATHS['p_dir'], f))]
original_files

['3_Crane.pst',
 '3_crane.insfile_data.csv',
 '3_crane.obs_data.csv',
 '3_crane.par_data.csv',
 '3_crane.pargp_data.csv',
 '3_crane.tplfile_data.csv',
 'custom_forward_run.py',
 'etf_S2.ins',
 'mult2model_info.csv',
 'p_aw_S2_0_constant.csv.tpl',
 'p_mad_S2_0_constant.csv.tpl',
 'p_ndvi_alpha_S2_0_constant.csv.tpl',
 'p_ndvi_beta_S2_0_constant.csv.tpl',
 'p_rew_S2_0_constant.csv.tpl',
 'p_swe_alpha_S2_0_constant.csv.tpl',
 'p_swe_beta_S2_0_constant.csv.tpl',
 'p_tew_S2_0_constant.csv.tpl',
 'params.csv',
 'swe_S2.ins']

Check out the files. We see the PEST++ control file, several csv files pointing to parameter information, our python run script, and .tpl and .ins files that spell out to PEST++ where to put the parameter data, and how to read the observations. The params.csv holds our default parameter values and intial estimates of soil parameters from the soils database.

### Critical Note

Looking at these files, we see we will have to make a couple changes:

1. The paths in `custom_forward_run.py` need to be changed. You will want to change the one in the `calibrate` package and copy it to the new 'pest' folder. Each new PEST++ build will copy `calibrate`'s `custom_forward_run.py` to your new/updated/debugged 'pest' folder.
2. The `tutorials/3_Crane/data/tutorial_config.toml` will need to be changed.
   - Turn the calibration on by setting `calibrate_flag = 1`.
   - Set the `calibration_folder = '{project_root}/pest/mult'`; this is where the parameter proposals will be placed).
   - Set the `initial_values_csv = '{project_root}/params.csv'`; these are the initial/default parameter values).


The PEST++ version 2 control file is succint; it delegates the work of detailing how to handle model output, observations, and parameter prosal file and format info to other files.

In [7]:
with open(PATHS['_pst'], 'r') as f: 
    print(f.read())

pcf version=2
* control data keyword
pestmode                                 estimation
noptmax                                 0
svdmode                                 1
maxsing                          10000000
eigthresh                           1e-06
eigwrite                                1
* parameter groups external
3_crane.pargp_data.csv
* parameter data external
3_crane.par_data.csv
* observation data external
3_crane.obs_data.csv
* model command line
python custom_forward_run.py
* model input external
3_crane.tplfile_data.csv
* model output external
3_crane.insfile_data.csv



Once we have to control file built, we will want to use the `build_localizer` function, that writes a `.loc` file matching the 'obersvations' from SNODAS and SSEBop to the parameters we want to tune them with. We only tune the SWE parameters `swe_alpha` and `swe_beta` using the SNODAS data, while we tune the other parameters using the SSEBop ETf data. The localizer matrix specifies that for PEST++.

Will will also run the `write_control_settings` that will change how many time the model runs.

In [8]:
build_localizer(PATHS['_pst'], ag_json=PATHS['prepped_input'])
write_control_settings(PATHS['_pst'], noptmax=3, reals=5)

noptmax:0, npar_adj:8, nnz_obs:3465
noptmax:3, npar_adj:8, nnz_obs:3465


The control file settings have been changed. The `noptmax` (number of optimization iterations) was increased to 3, with 5 model 'realizations' (runs) per cycle. Once we get the calibration running smoothly, increase the `reals` parameter to a larger number, perhaps 100. We can also see the addition of the `loc.mat` localizer file.

In [9]:
with open(PATHS['_pst'], 'r') as f: 
    print(f.read())

pcf version=2
* control data keyword
pestmode                                 estimation
noptmax                                 3
svdmode                                 1
maxsing                          10000000
eigthresh                           1e-06
eigwrite                                1
ies_localizer                  loc.mat
ies_num_reals                  5
* parameter groups external
3_crane.pargp_data.csv
* parameter data external
3_crane.par_data.csv
* observation data external
3_crane.obs_data.csv
* model command line
python custom_forward_run.py
* model input external
3_crane.tplfile_data.csv
* model output external
3_crane.insfile_data.csv



**Congratulations** if you've made it this far. There is a lot going on in this project, and staying organized while preparing up to harness a powerful tool like PEST++ is a significant achievement!

Let's see if we can improve SWIM through calibration. We're using multiprocessing; feel free to change `workers` to suit your machine.

Run the calibration launcher after modifying it to suit your paths:

In [10]:
from pprint import pprint
pprint(PATHS)

{'_pst': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.pst',
 'exe_': 'pestpp-ies',
 'ini_path': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/data/tutorial_config.toml',
 'input_ts_out': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/data/input_timeseries',
 'm_dir': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/master',
 'mult': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/mult',
 'obs': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/obs',
 'p_dir': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest',
 'prepped_input': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/data/prepped_input.json',
 'python_script': '/home/dgketchum/PycharmProjects/swim-rs/calibrate/custom_forward_run.py',
 'w_dir': '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/workers'}


In [11]:
workers = 8

run_pst(PATHS['p_dir'], PATHS['exe_'], PATHS['_pst'], num_workers=workers, worker_root=PATHS['w_dir'],
        master_dir=PATHS['m_dir'], verbose=True)

rmtree: /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/workers/worker_1
rmtree: /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/workers/worker_0
master:pestpp-ies /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.pst /h :5000 in /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/master


             pestpp-ies: a GLM iterative ensemble smoother

                   by the PEST++ development team


version: 5.2.7
binary compiled on Dec 12 2023 at 13:33:23

started at 01/05/25 22:39:07
...processing command line: ' pestpp-ies /home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.pst /h :5000'
...using panther run manager in master mode using port 5000

using control file: "/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.pst"
in directory: "/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/master"
on host: "dgketchum-r"

processing control file /home/dgketchum/PycharmProjects/swim-rs/tutorials

Error: [('/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.rns', '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/workers/worker_1/3_Crane.rns', "[Errno 2] No such file or directory: '/home/dgketchum/PycharmProjects/swim-rs/tutorials/3_Crane/pest/3_Crane.rns'")]

If it runs, you see a progress updater that will have something like `01/05 11:35:11 mn:0.16  runs(C5   |F0    |T0    ) agents(R1   |W0   |U0   ) 0`. The `C` stands for 'complete', and if it increases, PEST++ is running. Go get a coffee. 

Let's assume it didn't run.

### Debugging Tips

 - If you never saw the panther, then `pestpp-ies` was probably not executed. Make sure you can run `pestpp-ies` from the command line in any directory on your machine. You may need to point to the executable with a full path, like `/home/skywalker/software/pestpp-ies`, or a path that ends with the '.exe' extension, if on a Windows machine. In this case, you will need to update the `PATHS` dict above to ensure we are providing `run_pest.py`'s `run_pst` with the correct executable.
 - If you never saw the panther and got a Python error traceback, read it carefully. It's tricky to get the interface to work, as we need to launch `run_pest.py`, it needs to launch `pestpp-ies`, that launches `custom_forward_run.py`, which finally actually runs the model with `run/run_mp.py`.

   A good debugging approach is to start from the bottom up by getting `run_mp.optimize_fields` to run from arguments provided under `if __name__ == '__main__':` in `run_mp.py`. Then get `run_mp.py` to run by launching the `custom_forward_run.py` located in your 'pest' directory. Then try running `run/run_pest.py` with arguments provided under `if __name__ == '__main__':`. Trust a simpler way code flow that doesn't decrease flexibility is sought.

 - Try running the `pestpp-ies` commmand from the 'pest' folder. This runs the program in a single thread, and can rule out problems with the 'pest' folder's files and structure. If you can run this, the problem is likely with the `run/run_pest.py` function `run_pst`. Double check the paths and arguments. Try launching it from `run/run_pest.py` instead of from this notebook.
   
 - If you saw the panther, then `pestpp-ies` ran. Great. You are close. The traceback (message in the ouput) that traces your error is very informative, but the last error is likely not what you need to track down. It's common to see something like
    ```
    thread processing instruction file raised an exception: InstructionFile error in file 'swe_US-FPe.ins' : output file'pred/pred_swe_US-FPe.np' not found
    ```

This interrupted the PEST++ execution of the realization, but likely wasn't the true cause. The error of not finding SWIM's prediction in `pred_swe_US-FPe.np` is actually because SWIM never completed it's run, because SWIM itself has an error. If you look higher up the traceback, you might find a Python error, like 
    
    ```
    File "/home/yoda/PycharmProjects/swim-rs/swim/config.py", line 124, in read_config
    cal_files, mult_files = set(os.listdir(self.calibration_folder)), set(_files)
    FileNotFoundError: [Errno 2] No such file or directory: ''
    ```
 We see that we forgot to set the `calibration_folder` in the config file and Python raises an error when it sees only `''`, but expects a path to a directory.

These are just a few ideas. As always, the key to debugging is reading the hints in the traceback and moving up the code operation chain until the problem is found. Science says 9/10 errors are due to paths not being set correctly.
 


Once we get a successful run, we see we have many more files in the 'pest' directory, but what we want are the calibrated parameters we'll need to use to run SWIM in forecast mode (i.e., a calibrated run of the model). The should be in the 'pest' directory, though in cases theu may end up in 'master', the location where multiprocessing by pyemu of PEST++ was coordinated:

In [12]:
[f for f in sorted(os.listdir(PATHS['p_dir'])) if '.par.csv' in f]

['3_Crane.0.par.csv',
 '3_Crane.1.par.csv',
 '3_Crane.2.par.csv',
 '3_Crane.3.par.csv']

Make sure these exist. There is a parameter file for each optimization run, the intial '0' run, and the three optimization runs we specified with `noptmax`. Each file has a row for each realization, with columns having a parameter value for each tunable parameter. This is the valuable data we will examine in the next step.

This workflow benefits from a powerful machine; the higher number of workers you can employ, the faster the otpimization will run.